# **LINEARISATION UNIQUE DES TRAFICS SUR BORDEAUX METROPOLE**
> Traitements à mettre en oeuvre

In [118]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\martin.schoreisz\git\Lin_uniq_BdxM\Lin_uniq_BdxM\Traitements')
import Connexion_Transfert as ct
import geopandas as gp
import pandas as pd
import numpy as np
from shapely.geometry import MultiPoint, Polygon, box
import re
from difflib import get_close_matches,SequenceMatcher
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from collections import Counter
from Outils import creer_graph, plus_proche_voisin,nb_noeud_unique_troncon_continu

from Repartition_trafic import corresp_noeud_mmm,corresp_noeud_rhv,appariement_noeud_mmm_fv,noeud_fv_ligne_ss_trafic
from Simplifier_Rdpt import creer_dico_noeud_rdpt,simplifier_noeud_rdpt,maj_graph_rdpt

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 20)

from Base_BdTopo.Import_outils import *
from Base_BdTopo.Rond_points import *
from Base_BdTopo.Regroupement_correspondance import *
from Base_BdTopo.Troncon_elementaire import *
from Base_BdTopo.Troncon_base import *
from Base_BdTopo.Gestion_2_chaussee import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1 Regrouper les troncons du filaire de voie
>Les troncons du filaire de voie sont regroupés d'abord uniquement avec les troncons de cat_rhv égal à 1,2,3,61,62,63, pour pouvoir réaliser des troncon de trafic homogènes tous reliés entre eux.
<br> Ensuite, un deuxième regroupement comprenant l'ensemble des troncons de cat_rhv permetd'affiner le diagnostic

> ### 1.1 Import du fichier final
le fichier shape est vérifié à la main car l'affectation automatique n'est pas 100% fiable

In [53]:
# Pour info : lire le graph issu de la methode decrite dessous
with ct.ConnexionBdd('local_otv') as c :
    graph_filaire_123 = gp.read_postgis('select * from linearisation_bm.graph_rhv_123', c.connexionPsy)
    graph_filaire_123_vertex = gp.read_postgis('select id,cnt,chk,ein,eout,the_geom as geom from linearisation_bm.graph_rhv_123_vertices_pgr', c.connexionPsy)
    graph_filaire = gp.read_postgis('select * from linearisation_bm.graph_rhv_complet', c.connexionPsy)

In [54]:
gdf_rhv_groupe_123=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_groupe_123_v2.shp').merge(
    graph_filaire_123[['ident','source','target']], on='ident')
gdf_rhv_groupe=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\rhv_grp.shp').merge(
    graph_filaire[['ident','source','target']], on='ident')

>### 1.2 Méthode
 pour information

In [45]:
# 1. import du fichier de filaire de base
gdf_rhv=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\Bdx-Metro\Filaire_voie\FV_TRONC_L.shp')
# 2. mise en forme
gdf_rhv.columns=[a.lower() for a in gdf_rhv.columns] #nom de colonne en minuscule
gdf_rhv.rename(columns={'gid':'id','nom_voie' : 'numero'},inplace=True)
gdf_rhv=gdf_rhv.loc[~gdf_rhv.id.isna()].copy()
gdf_rhv['id_ign']=gdf_rhv.ident.apply(lambda x : 'TRONROUT'+str(x))
gdf_rhv=gdf_rhv.loc[~gdf_rhv.cat_rhv.isin(['98','99'])].copy()
gdf_rhv['nature']=gdf_rhv.apply(lambda x : 'Route à 2 chaussées' if x['rgraph_dbl']==0 else 'Route à 1 chaussée', axis=1)
gdf_rhv['nature']=gdf_rhv.apply(lambda x : 'Bretelle' if x['cat_dig']=='8' else x['nature'], axis=1)
gdf_rhv['numero']=gdf_rhv.apply(lambda x : 'Bretelle '+x['numero'] if x['cat_dig']=='8' else x['numero'], axis=1)
gdf_rhv['sens']=gdf_rhv.apply(lambda x : 'Direct' if x['rgraph_dbl']==0 else 'Double', axis=1)
gdf_rhv['codevoie_d']='NR'
gdf_rhv['importance']=gdf_rhv['cat_dig']
gdf_rhv['id']=gdf_rhv.id.apply(lambda x : int(x))
#gdf_rhv.loc[gdf_rhv.nom_voie.isna()] #verif des nom_voie null : 2 lignes, cf traitements plus loins
#filtrer les voies cylcables et autres
gdf_rhv_filtre=gdf_rhv.loc[~gdf_rhv.cat_dig.isin(['6','7','9','10'])].copy()

# 3. creer le graph en bdd
#creer_graph(gdf_rhv_filtre.loc[gdf_rhv_filtre.cat_rhv.isin(['1','2','3','33',61','62','63'])],
            #'local_otv',schema='linearisation_bm',table='graph_rhv_123',table_vertex='graph_rhv_123_vertices_pgr')
creer_graph(gdf_rhv_filtre,'local_otv',schema='linearisation_bm',table='graph_rhv_complet',table_vertex='graph_rhv_complet_vertices_pgr')

In [258]:
# 4. initialiser les paramètre de regroupement
df=import_donnes_base('local_otv','linearisation_bm', 'graph_rhv_complet','graph_rhv_complet_vertices_pgr')
df2_chaussees=df.loc[df.nature.isin(['Autoroute', 'Quasi-autoroute', 'Route à 2 chaussées'])]
df_avec_rd_pt,carac_rd_pt,lign_entrant_rdpt=identifier_rd_pt(df)
df_lignes=df_avec_rd_pt.set_index('id_ign')#mettre l'id_ign en index
bretelle=df_avec_rd_pt.loc[df_avec_rd_pt['nature']=='Bretelle'].copy()
bretelle_tri=bretelle.loc[bretelle.length.sort_values(ascending=False).index.tolist()].id_ign.tolist()
sans_bretelle=df_avec_rd_pt.loc[df_avec_rd_pt['nature']!='Bretelle'].copy()
list_sans_bretelle=sans_bretelle.id_ign.tolist()
list_tri_longueur=sans_bretelle.loc[sans_bretelle.length.sort_values(ascending=False).index.tolist()].id_ign.tolist()
list_id_ign=bretelle_tri+list_tri_longueur

c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [45]:
# 5. appel fonction de grouepement pour creer des idtronc pour uniquement voies de 1 à 3 et d'un autre coté pour toute les voies
df_affectation, dico_erreur, lignes_traitees, lignes_non_traitees=regrouper_troncon(
    list_id_ign, df_avec_rd_pt, carac_rd_pt,df2_chaussees,[])

# 6. mise en forme et export
gdf_rhv_groupe=gdf_rhv_filtre.merge(df_affectation, left_on='id_ign', right_on='id', how='left')
gdf_rhv_groupe.to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\rhv_grp_v2.shp')
#gdf_rhv_groupe_123=gdf_rhv_filtre.merge(df_affectation, left_on='id_ign', right_on='id', how='left')
#gdf_rhv_groupe_123.to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_groupe_123.shp')

In [265]:
#pour test sur une ligne
df_affectation2, dico_erreur2, lignes_traitees2, lignes_non_traitees2=regrouper_troncon(
    ['TRONROUT9919'], df_avec_rd_pt, carac_rd_pt,df2_chaussees,[])

0 2019-12-12 11:48:54.415530 nb lignes traitees : 0
pas de parrallele trouvee pour les troncons ['TRONROUT9919']


## 1. MISE EN FORME DES COMPTAGES PONCTUELS
Pour ces comptages l'idée c'est de réussir à regrouper les comptages par un idtronc issu de la detection des troncon elementaire. ça a du sens car pour un comptage ponctuel normalement les 2 sens de cicru sont sur un troncon unique. DE plus, les ens circu, noms de voie et autres attributs permettant de regrouper les comptages sont de qualité trop variable pour s'appuyer uniquement dessus. et enfin, peu de 2*2 vois sont meusrés

In [55]:
#import des données
cpt_pct=pd.read_excel(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\Bdx-Metro\comptages\base_comptages_routiers_v4.xlsx')
gdf_rhv_groupe['geom_src']=gdf_rhv_groupe.geometry
cpt_pct.columns=[a.lower() for a in cpt_pct.columns]
cpt_pct = gp.GeoDataFrame(cpt_pct, geometry=gp.points_from_xy(cpt_pct.longitude, cpt_pct.latitude))
cpt_pct.crs = {'init' :'epsg:4326'}
cpt_pct_l93=cpt_pct.to_crs({'init': 'epsg:2154'})
cpt_pct_l93['x_l93']=cpt_pct_l93.geometry.apply(lambda x : x.x)
cpt_pct_l93['y_l93']=cpt_pct_l93.geometry.apply(lambda x : x.y)

#mise en forme
cpt_pct_l93['nom_voie']=cpt_pct_l93.nom_voie.apply(lambda x : re.sub(('é|è|ê'),'e',x.lower().strip()))
cpt_pct_l93['sens_unique']=cpt_pct_l93.sens_circulation.apply(lambda x : True if SequenceMatcher(None,' '.join(x.split(' ')[:2]).lower(),'sens unique').ratio()>0.8 else False)
cpt_pct_l93['type_voie']=cpt_pct_l93.nom_voie.apply(lambda x : x.split(' ')[0])
cpt_pct_l93['suffix_nom_voie']=cpt_pct_l93.nom_voie.apply(lambda x : ' '.join(x.split(' ')[1:]).lower())
cpt_pct_l93['date_max_cptg']=cpt_pct_l93.observation.apply(lambda x : pd.to_datetime(x.split(' au ')[1],dayfirst=True))
cpt_pct_l93['sens_circulation']=cpt_pct_l93.sens_circulation.apply(lambda x : re.sub(('é|è|ê'),'e',x.strip().lower()))
cpt_pct_l93['observation']=cpt_pct_l93.observation.apply(lambda x : x.strip().lower())

In [56]:
#cluster à 200m
data_test_clust=[[x, y] for x, y in zip(cpt_pct_l93.x_l93.tolist(), cpt_pct_l93.y_l93.tolist())]
db = DBSCAN(eps=200, min_samples=2).fit(data_test_clust)
labels = db.labels_
cpt_pct_l93['n_cluster']=labels

#voies avec nom_proches
cross_join_ncluster=cpt_pct_l93[['ident','nom_voie','type_voie','suffix_nom_voie', 'sens_circulation','annee','n_cluster']].merge(
   cpt_pct_l93[['ident','nom_voie','type_voie','suffix_nom_voie', 'sens_circulation','annee','n_cluster']], on='n_cluster') #avoir toutes les relations internoms possibles
cross_join_ncluster['comp_nom_voie']=cross_join_ncluster.apply(lambda x : SequenceMatcher(None,x['suffix_nom_voie_x'], x['suffix_nom_voie_y']).ratio(), axis=1)#affecter une note a cahque relation
voie_nom_proches=cross_join_ncluster.loc[(cross_join_ncluster['comp_nom_voie']>0.85) & (cross_join_ncluster['type_voie_x']==cross_join_ncluster['type_voie_y'])
                        ].sort_values(['n_cluster','ident_x'])#ne conserver qque les relations bien notees
voie_nom_proches['id_grp_nom_voie']=voie_nom_proches.ident_x.rank(method='dense')#ajouter un id 
corresp_voies=voie_nom_proches.drop_duplicates('ident_y')
grp_nom_voie=cpt_pct_l93.merge(corresp_voies[['ident_y','id_grp_nom_voie']].rename(columns={'ident_y':'ident'}), on='ident')#jointure sur id de depart

#grouper par periode
grp_period=grp_nom_voie.copy()
grp_period['date_max_cptg']=grp_period.observation.apply(lambda x : pd.to_datetime(x.split(' au ')[1],dayfirst=True))
grp_period['id_period']=grp_period.date_max_cptg.rank(method='dense')
grp_period['indicefinal']=grp_period.n_cluster+(grp_period.id_grp_nom_voie*(1000))+(grp_period.id_period*10000)
grp_period[[a for a in grp_period.columns if a!='date_max_cptg']].to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\cluster_comptg_pctl_grp_period.shp')

In [57]:
#regarder le nb de ligne relative à chaque comptage : siune ligne est relative à 2 comptage qui ont le mm indice final
#i.e relatif à la mme rue, à la mm periode, sur des sens différents, on fait la somme
#si il y a plus de 2 comptages : il faut récupérer les plus recents : si 2 on fait la somme, si 1 on prends la valeur

#rappatrier le numero d'id_tronc pour chaque comptage
#trouver la distance min à chaque objet ligne"du rhv groupe
grp_troncon_temp=cpt_pct_l93.copy()
grp_troncon_temp.geometry=grp_troncon_temp.buffer(20)#passer la geom en buffer
intersct_buff_20=gp.sjoin(grp_troncon_temp,gdf_rhv_groupe,how='left',op='intersects')
intersct_buff_20.geometry=grp_period.geometry#repasser la geom en point
intersct_buff_20=intersct_buff_20.merge(gdf_rhv_groupe[['ident','geometry']], left_on='ident_right', right_on='ident')
intersct_buff_20['dist_pt_ligne']=intersct_buff_20.apply(lambda x : x['geometry_x'].distance(x['geometry_y']), axis=1) #définir la disance entre les points et ligne
joint_dist_min=intersct_buff_20.loc[intersct_buff_20.groupby('ident_left')['dist_pt_ligne'].transform(min)==intersct_buff_20['dist_pt_ligne']][['ident_left','idtronc','numero']].rename(
    columns={'ident_left':'ident'}).copy()#ne garder que la ligne la plus proche
grp_troncon=grp_period.merge(joint_dist_min, on='ident',how='left')#df finale
grp_troncon[[a for a in grp_troncon.columns if a!='date_max_cptg']].to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\cluster_comptg_pctl_grp_troncon.shp')

c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [58]:
#annee la plus recente
anne_recente=grp_troncon.loc[grp_troncon.groupby('idtronc')['date_max_cptg'].transform(max)==grp_troncon['date_max_cptg']].copy()

In [59]:
#idtronc ok (avec 1 ou 2 pt de comptage)
idtronc_grp=anne_recente.groupby('idtronc').ident.count()
idtroncOkTmjo=anne_recente.loc[anne_recente.idtronc.isin(idtronc_grp.loc[idtronc_grp<3].index.tolist())][['ident','idtronc','sens_circulation','tmjo_tv']].copy()

In [60]:
#isoler les idtronc supportant + de 2 pt de comptages
idtronc_sup2=idtronc_grp.loc[idtronc_grp>2].copy()
#trouver les points correspondants
pt_sup2=anne_recente.loc[anne_recente['idtronc'].isin(idtronc_sup2.index.tolist())].copy()
#ajouter attribut qui traduit le nb de valeurs différentes de sens circulation
def nb_sens_circu(idtronc) :
    return len(pt_sup2.loc[pt_sup2['idtronc']==idtronc].sens_circulation.unique())  
pt_sup2['nb_sens_circu']=pt_sup2.apply(lambda x : nb_sens_circu(x['idtronc']), axis=1)
#pour les points a 2 sens de circu : on prend la valeur max des 2 : 
ptSup2SensCircu2=pt_sup2.loc[pt_sup2['nb_sens_circu']==2].copy()
ptSup2SensCircu2.drop_duplicates(['nom_voie','sens_circulation','tmjo_tv','observation'],inplace=True)#qq points ont des ident différents mais sont les mêmes
ptSup2SensCircu2.drop_duplicates(['sens_circulation','tmjo_tv','observation'],inplace=True)#qla mm que la précédente, mais je ne sais pas pourquoi l'ajout de nom_voie fait bugger le drop duplicates pour les ident  716,717,975,976
ptSup2SensCircu2OkTmjo=ptSup2SensCircu2.groupby(['idtronc','sens_circulation'])['tmjo_tv'].max().reset_index().merge(
    ptSup2SensCircu2[['idtronc','sens_circulation','tmjo_tv','ident']], on=['idtronc','sens_circulation','tmjo_tv'], how='left')

In [61]:
#pour les autres point
ptSup2SensCircuSup2=pt_sup2.loc[pt_sup2['nb_sens_circu']>2][['ident','idtronc','sens_circulation','tmjo_tv','indicefinal']].sort_values(['idtronc','indicefinal','sens_circulation']).copy()

#filtre des points dont le sens circul est le mm ou qui sont isole
list_ident, list_idtronc, list_indicefinal, list_senscircu=(ptSup2SensCircuSup2.ident.tolist(),ptSup2SensCircuSup2.idtronc.tolist(),
                                                            ptSup2SensCircuSup2.indicefinal.tolist(),ptSup2SensCircuSup2.sens_circulation.tolist())
list_new_ident=list_ident
for i in range(len(list_ident)) : 
    if i<len(list_ident)-1 :
        if list_idtronc[i+1]==list_idtronc[i] : 
            if list_indicefinal[i+1]==list_indicefinal[i] :
                if SequenceMatcher(None,list_senscircu[i+1], list_senscircu[i]).ratio()>0.75 : 
                    list_new_ident[i+1]=list_ident[i]
    else : 
        if list_idtronc[i]==list_idtronc[i-1] : 
            if list_indicefinal[i]==list_indicefinal[i-1] :
                if SequenceMatcher(None,list_senscircu[i], list_senscircu[i-1]).ratio()>0.75 : 
                    list_new_ident[i]=list_ident[i-1]
ptSup2SensCircuSup2['ident_final']=np.array(list_ident)
#filtrer les points isoles
ptSup2SensCircuSup2_filtre=ptSup2SensCircuSup2.groupby(['idtronc','indicefinal']).nunique()[['ident']].reset_index()
ptSup2SensCircuSup2=ptSup2SensCircuSup2.loc[ptSup2SensCircuSup2.indicefinal.isin(ptSup2SensCircuSup2_filtre.loc[ptSup2SensCircuSup2_filtre['ident']>1].indicefinal.
                                                                             tolist())].copy()
#filtrer les points qui ont des nom des sens circu proches
ptSup2SensCircuSup2_filtre=ptSup2SensCircuSup2.loc[ptSup2SensCircuSup2['tmjo_tv']==
                                                          ptSup2SensCircuSup2.groupby('ident_final')['tmjo_tv'].transform(max)].copy()
#filtrer les points qui sont égaux 
ptSup2SensCircuSup2_filtre=ptSup2SensCircuSup2_filtre.loc[ptSup2SensCircuSup2_filtre['ident']==ptSup2SensCircuSup2_filtre['ident_final']].copy()
#filtrer les points qui présente toujours plus de 2 identifiant (i.e pb denomination ou pb référentiel ou pb tronc_elementaire)
grp=ptSup2SensCircuSup2_filtre.groupby('idtronc').nunique()[['ident']].reset_index()
pt_non_affectes=grp.loc[grp['ident']>2]
ptSup2SensCircuSup2Oktmjo=ptSup2SensCircuSup2_filtre.loc[ptSup2SensCircuSup2_filtre.idtronc.isin(grp.loc[grp['ident']<3].idtronc.tolist())][['ident','idtronc','sens_circulation','tmjo_tv']].copy()

In [62]:
affect_finale=pd.concat([ptSup2SensCircuSup2Oktmjo,idtroncOkTmjo,ptSup2SensCircu2OkTmjo],axis=0, sort=False)

In [63]:
#qq modifs manuelle pour grouper par paire les comptages ponctuels: 
affect_finale.loc[affect_finale['ident']==606,'idtronc']=affect_finale.loc[affect_finale['ident']==605].idtronc.values[0]

In [64]:
affect_finale.loc[affect_finale['ident']==1502]

,ident,idtronc,sens_circulation,tmjo_tv
1501,1502,10986.0,vers l'avenue du marechal leclerc,6703


## 2. MISE EN FORME DES COMPTAGES PERMANENTS
Pour ces compteurs on va regrouper en séparant les siredo des boucles liées au système gertrude. Pour les siredo pas de pb, pour les autre sc'est plus compliqué car les références de localiusation varient parfois. Il y a donc une phase de regroupement manuel à la fin

> ### 2.1 Import du fichier final

In [65]:
#import fichier corrige_final
cpt_perm_final=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\cpt_perm_groupe.shp')

> ### 2.1 Méthode
Pour info

In [5]:
#creer un gdf avec la fusion des points et des données de comptages non nulles
cpt_brut=pd.read_excel(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\Bdx-Metro\comptages\comptage_trafic_2017.xlsx')
cpt_brut.columns=[a.lower() for a in cpt_brut.columns]
cpt_brut=cpt_brut.loc[~cpt_brut.mjo_val.isna()].copy()
pt_cpt_perm=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\Bdx-Metro\comptages\PC_CAPTE_P.shp')
pt_cpt_perm.columns=[a.lower() for a in pt_cpt_perm.columns]
pt_cpt_perm=pt_cpt_perm.merge(cpt_brut, on='ident')
pt_cpt_perm['nom_voie']=pt_cpt_perm.nom_voie.apply(lambda x : re.sub(('é|è|ê'),'e',x.lower().replace('  ',' ')))#mise en forme nom de voie

#regrouper les comptages
#un compteur peut caracteriser 1 ligne, ou 2 compteur 1 ligne ou 2 compteurs 2 lignes, dc les compteurs ne vont pas toujours par deux

#regrouper geographiquement
pt_cpt_perm['x_l93']=pt_cpt_perm.geometry.apply(lambda x : x.x)
pt_cpt_perm['y_l93']=pt_cpt_perm.geometry.apply(lambda x : x.y)
data_test_clust=[[x, y] for x, y in zip(pt_cpt_perm.x_l93.tolist(), pt_cpt_perm.y_l93.tolist())]
db = DBSCAN(eps=200, min_samples=2).fit(data_test_clust)
labels = db.labels_
pt_cpt_perm['n_cluster']=labels

#correspondance nom_rue
#analyser les noms de commune : 
#grp_nom_voie.nom_voie.apply(lambda x : x.split()[0]).unique()#liste des permiers mots
#grp_nom_voie.loc[grp_nom_voie.nom_voie.apply(lambda x : x.split()[0]=='carbon')].nom_voie.unique() #test des valeusr
dico_commune={'le bouscat','le haillan','bordeaux','pessac','talence', 'merignac','st medard','carbon blanc',
       'begles', 'eysines','villenave d\'ornon','bruges','gradignan','bassens','cenon','floirac','pessa','lormont', 'artigues'}
dico_type_voie={'allee' : ['all.','allee', 'allees'],'avenue': ['av', 'av.', 'avenue'], 'boulevard' : ['blvd'],
               'bretelle' : 'bretelle', 'cours' : ['crs','cours'], 'cote' : ['côte'], 'place':['pl'], 'route':['route', 'rte'],'rue':['rue'],
               'voie':['voie']}
def decoupe_nom_voie(nom_voie) : 
    for c in dico_commune : 
        if nom_voie[:len(c)]==c :
            commune=nom_voie[:len(c)]
            nom_voie=nom_voie[len(c):].strip()
    for k, v in dico_type_voie.items() : 
        if any([a for a in nom_voie.split() if a in v])  : 
            caractere=[a for a in nom_voie.split() if a in v][0]
            type_voie=k
            nom_voie=' '.join([n for n in nom_voie.split() if n!=caractere])
            break
    else : type_voie=None
    for test in ['avant','apres'] :
        if test in nom_voie : 
            return nom_voie.split(test)[0].strip(), test,nom_voie.split(test)[1].strip(), commune, type_voie
    else : return nom_voie, nom_voie,nom_voie,commune,type_voie

pt_cpt_perm['nom_rue']=pt_cpt_perm.nom_voie.apply(lambda x : decoupe_nom_voie(x)[0])
pt_cpt_perm['localisant']=pt_cpt_perm.nom_voie.apply(lambda x : decoupe_nom_voie(x)[1])
pt_cpt_perm['rue_refer']=pt_cpt_perm.nom_voie.apply(lambda x : decoupe_nom_voie(x)[2])
pt_cpt_perm['commune']=pt_cpt_perm.nom_voie.apply(lambda x : decoupe_nom_voie(x)[3])
pt_cpt_perm['type_voie']=pt_cpt_perm.nom_voie.apply(lambda x : decoupe_nom_voie(x)[4])
pt_cpt_perm.to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\pt_cpt_perm.shp')

>>#### 2.2.1 SIREDO

In [ ]:
siredo=pt_cpt_perm.loc[pt_cpt_perm['type']=='SIREDO'].copy()#isoler siredo
cross_join=siredo.assign(key=1).merge(siredo.assign(key=1), on='key').drop('key',axis=1)#calcul des distances
cross_join['distance']=cross_join.apply(lambda x : x['geometry_x'].distance(x['geometry_y']),axis=1)
cross_join_filtre=cross_join[[c for c in cross_join.columns if c[-2:]=='_x']+['distance','ident_y']].rename(columns=
              {c : c[:-2] for c in cross_join.columns if c[-2:]=='_x'}).copy()#filtre des attribut
cross_join_filtre=cross_join_filtre.loc[(cross_join_filtre['ident']!=cross_join_filtre['ident_y'])].copy()
siredo_proches=cross_join_filtre.loc[(cross_join_filtre.groupby('ident')['distance'].transform(min)==
                                      cross_join_filtre['distance'])].copy()#plus proche voisin
siredo_proches['id_grpsiredo']=siredo_proches.reset_index().index
id_siredo_proches=(pd.concat([siredo_proches[['ident','ident_y', 'id_grpsiredo']],
           siredo_proches[['ident_y','ident', 'id_grpsiredo']].rename(columns={
    'ident_y':'ident', 'ident':'ident_y'})],sort=False).sort_values('ident').
    reset_index().drop_duplicates('index').drop_duplicates(['ident','ident_y']).drop('index',axis=1))
cle_assoc_siredo=pd.concat([id_siredo_proches[['ident','id_grpsiredo']],id_siredo_proches[['ident_y','id_grpsiredo']].rename(columns={
    'ident_y':'ident'})]).sort_values('id_grpsiredo')

>>#### 2.2.2 Gertrude
> au final ce n'est pas très efficace, et un renseignement des regroupement a la ain aurait été plus rapide. il sera important si de nouvelles stations sont posées de fournir de suite un identifiant de regroupement

In [23]:
gertrude=pt_cpt_perm.loc[pt_cpt_perm['type']=='BOUCLE'].copy()#isoler les boucles
gertrude['x']=gertrude.geometry.apply(lambda x : x.x)
gertrude['y']=gertrude.geometry.apply(lambda x : x.y)

In [ ]:
#si c'est sur le mm idtronc on regroupe

#rappatrier le numero d'id_tronc pour chaque comptage
#trouver la distance min à chaque objet ligne"du rhv groupe
grp_troncon_temp=gertrude.copy()
grp_troncon_temp.geometry=grp_troncon_temp.buffer(20)#passer la geom en buffer
intersct_buff_20=gp.sjoin(grp_troncon_temp,gdf_rhv_groupe,how='left',op='intersects')
intersct_buff_20.geometry=gertrude.geometry#repasser la geom en point
intersct_buff_20=intersct_buff_20.merge(gdf_rhv_groupe[['ident','geometry']], left_on='ident_right', right_on='ident')
intersct_buff_20['dist_pt_ligne']=intersct_buff_20.apply(lambda x : x['geometry_x'].distance(x['geometry_y']), axis=1) #définir la disance entre les points et ligne
joint_dist_min=intersct_buff_20.loc[intersct_buff_20.groupby('ident_left')['dist_pt_ligne'].transform(min)==intersct_buff_20['dist_pt_ligne']][['ident_left','idtronc','numero']].rename(
    columns={'ident_left':'ident'}).copy()#ne garder que la ligne la plus proche
grp_troncon=gertrude.merge(joint_dist_min, on='ident',how='left')#df finale

#regrouper
grp_idtronc=grp_troncon.groupby('idtronc')['ident'].agg(lambda x : tuple(x))
grp_idtronc=grp_idtronc.loc[grp_idtronc.apply(lambda x : len(x)>1)].reset_index().copy()
grp_idtronc['id_grp']=grp_idtronc.ident.rank()
#mettre en forme
grp_idtronc=pd.DataFrame([(a,g) for i,g in zip(grp_idtronc.ident.tolist(),grp_idtronc.id_grp.tolist()) for a in i], columns=['ident','id_grp'])
grp_final=grp_idtronc.copy()

In [292]:
# trouver les points qui restent et préparer un regrouepement par commune, nom de rue, distance et se,s

#points qui restent
gertrude_etape2=gertrude.loc[~gertrude['ident'].isin(grp_idtronc.ident.tolist())].copy()
#comparer les noms
cross_join=gertrude_etape2[['gid','geometry','ident','nom_voie', 'sens_cir','commune', 'nom_rue', 'localisant','rue_refer','type_voie']].assign(key=1).merge(
    gertrude_etape2[['gid','geometry','ident','nom_voie', 'sens_cir','commune', 'nom_rue', 'localisant','rue_refer','type_voie']].assign(key=1), on='key').drop('key',axis=1)#calcul des distances
cross_join['distance']=cross_join.apply(lambda x : x['geometry_x'].distance(x['geometry_y']),axis=1)
cross_join['comp_nom_rue']=cross_join.apply(lambda x : SequenceMatcher(None,x['nom_rue_x'], x['nom_rue_y']).ratio(), axis=1)#affecter une note a cahque relation

#vérifier que les sens de cicru sont cohérents et ne garder que les lignes avec un sens ok, un nom de voie proche et une distance proche
gertrude_join=cross_join.loc[(cross_join['comp_nom_rue']>0.6) & (cross_join['distance']<1000) & (cross_join['commune_x']==cross_join['commune_y']) & 
                             (cross_join['type_voie_x']==cross_join['type_voie_y'])].copy()
#on ajoute un attribut de verif du sens
def verif_sens(s1,s2) : 
    for sens in [['Sens Sortant', 'Sens Entrant'],['Sens N > S', 'Sens S > N'], ['Sens Nord > Sud', 'Sens Sud > Nord'],['Nord > Sud', 'Sud > Nord'],
                ['Boulevards Nord > Sud', 'Boulevards Sud > Nord']] :
        if s1 in sens and s2 in sens : 
            if s1!=s2 : 
                return True
        else : 
            continue
    return False
gertrude_join['valid_sens']=gertrude_join.apply(lambda x : verif_sens(x['sens_cir_x'],x['sens_cir_y']),axis=1)  
gertrude_join['id_grp']=gertrude_join.sort_values('ident_x').ident_x.rank(method="min")
gertrude_grp=gertrude_join.loc[gertrude_join['valid_sens']][['ident_x','ident_y','distance', 'comp_nom_rue', 'valid_sens','id_grp']].sort_values('ident_x')

In [228]:
#obtenir une liste des ident qui se regroupent
list_id_grp=pd.concat([gertrude_grp[['ident_x','id_grp']],gertrude_grp[['ident_y','id_grp']].rename(columns={'ident_y' : 'ident_x'})]).drop_duplicates([
    'ident_x','id_grp']).groupby('id_grp').ident_x.agg(lambda x : set(sorted(tuple(x)))).tolist()
dico={}
for e,c in enumerate(gertrude_grp.ident_x.tolist()) : 
    for l in list_id_grp : 
        if c in l :
            #print('c dans l','c : ',c,'l :',l)
            if c not in dico.keys() :
                dico[c]=list(l)
            else : 
                dico[c]+=list(l)
list_id_grp_temp=set([ v for v in {k : tuple(set(sorted(dico[k]))) for k in dico.keys()}.values()])
list_id_grp_final=set(v for v in {c :b for c in gertrude_grp.ident_x.tolist() 
 for b in [a for a in list_id_grp_temp if c in a] 
 if len(b) == max([len(c) for c in [a for a in list_id_grp_temp if c in a]])}.values())

In [281]:
#affecter un identifiant
def groupe_pt_gertrude_etape2(df_local) : 
    """
    regrouper les points à partir d'une liste d'ident selon l'ordre d'apparition 
    """
    limite_box=MultiPoint([(a.x,a.y) for a in df_local.itertuples()]).bounds
    plg_long, plg_larg=limite_box[2]-limite_box[0],limite_box[3]-limite_box[1]
    if max(plg_long,plg_larg)==plg_long : 
        df_local['next_sens']=df_local.sort_values('x').sens_cir.shift(-1,fill_value='NC')
        df_local['prev_sens']=df_local.sort_values('x').sens_cir.shift(1,fill_value='NC')
    else : 
        df_local['next_sens']=df_local.sort_values('y').sens_cir.shift(-1,fill_value='NC')
        df_local['prev_sens']=df_local.sort_values('y').sens_cir.shift(1,fill_value='NC')
    df_local['sens_comp']=df_local.apply(lambda x : x.prev_sens if x.next_sens=='NC' else x.next_sens,axis=1)
    df_local['verif_sens']=df_local.apply(lambda x : verif_sens(x['sens_cir'],x['sens_comp']),axis=1)
    df_local['id_grp']=-99
    i=0
    while i < len(df_local)-1:
        if df_local.iloc[i].verif_sens : 
            df_local.loc[df_local.index.isin(df_local.iloc[i:i+2].index.tolist()),'id_grp']=i
            i+=2
        else : 
            i+=1
    return

# à partior de la liste des ident groupé : 
grp_final2=grp_final.copy()
# on selectionne ces points dans la df
increment=grp_final2.id_grp.max()
for l in list_id_grp_final : 
    #if 'Z8CT13' not in l : continue
    df_local=gertrude_etape2.loc[gertrude_etape2.ident.isin(l)].copy()
    if len(df_local)==1:
        continue
    if len(df_local)==2 : 
        if (gertrude_join.loc[(gertrude_join.ident_x.isin(l)) & (gertrude_join['ident_x'] != gertrude_join['ident_y'])].valid_sens).all() : 
            increment+=1
            df_local['id_grp']=increment
            grp_final2=pd.concat([grp_final2,df_local[['ident','id_grp']]],sort=False)
    else :
        groupe_pt_gertrude_etape2(df_local)
        increment+=grp_final2.id_grp.max()
        df_local.loc[df_local['id_grp']!=-99,'id_grp']=df_local['id_grp']+increment
        grp_final2=pd.concat([df_local[['ident','id_grp']],grp_final2],sort=False)

In [317]:
#rappatrier l'id_grp sur les données de base et export pour modif manuelle
gertrude.merge(grp_final2, on='ident',how='left').to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gertrude.shp')

#import des modifs manuelles et mise en forme finale
gertrude_manu=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gertrude.shp')
gertrude_manu.loc[(gertrude_manu.id_grp.isna()) | (gertrude_manu.id_grp==-99),'id_grp']=gertrude_manu.loc[(gertrude_manu.id_grp.isna()) | (gertrude_manu.id_grp==-99),'id_grp'].reset_index().index + gertrude_manu.id_grp.max()
gertrude_manu.loc[gertrude_manu.type_grp.isnull(),'type_grp']='auto'
#export pour dernière modif manuelle
gertrude_manu.to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gertrude_final.shp')

#fichier final
gertrude_fin=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gertrude_final.shp')
#concat avec les siredo
cle_assoc_siredo['id_grpsiredo']=cle_assoc_siredo['id_grpsiredo']+gertrude_fin.id_grp.max()
cpt_perm_final=pd.concat([gertrude_fin,siredo.merge(cle_assoc_siredo, on='ident').rename(columns={'id_grpsiredo':'id_grp'})],axis=0, sort=False)
cpt_perm_final.to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\cpt_perm_groupe.shp')

# 3. Affecter du trafic au filaire de voie

> ### On commence par affecter le trafic aux troncons des catégories 1,2,3 à partirdes comptages permanents

In [66]:
#trouver la ligne la plus proche

lgn_proche_perm=cpt_perm_final.merge(plus_proche_voisin(cpt_perm_final,gdf_rhv_groupe,10,'ident','ident'),left_on='ident', right_on='ident_left',how='left').merge(
    gdf_rhv_groupe[['ident','cat_rhv','rgraph_dbl','idtronc']], left_on='ident_right', right_on='ident', how='left').rename(columns={'ident_right':'ident_lgn'})

c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [67]:
#calculer le trafic total par point de comptage permanent
def calcul_tmjo_2sens_perm(tmjo, rgraph_dbl, nb_cpt, df,id_grp,nom_attr_trafic,nom_attr_id_grp,id_cpt,nom_attr_id_cpt) :
    """calculer le tmja 2 sens en fonction du nb de compteur et du sens unique ou non"""
    if nb_cpt==2 : 
        return df.loc[df[nom_attr_id_grp]==id_grp][nom_attr_trafic].sum(), df.loc[df[nom_attr_id_grp]==id_grp].groupby(nom_attr_id_grp)[nom_attr_id_cpt].agg(
            lambda x : tuple(x)).values[0]
    else : 
        if rgraph_dbl==1 : 
            return tmjo*2, (id_cpt,)
        else : 
            return tmjo, (id_cpt,)

In [68]:
lgn_proche_attr_sup=lgn_proche_perm.merge(lgn_proche_perm.groupby('id_grp')['gid'].nunique(), left_on='id_grp',
                    right_index=True).rename(columns={'gid_y':'nb_cpt'})

In [69]:
lgn_proche_attr_sup['tmjo_2_sens']=lgn_proche_attr_sup.apply(lambda x : calcul_tmjo_2sens_perm(x['mjo_val'],x['rgraph_dbl'],
                                                                                          x['nb_cpt'], lgn_proche_attr_sup,x['id_grp'],
                                                                                          'mjo_val','id_grp',x['ident_x'],'ident_x')[0],axis=1)
lgn_proche_attr_sup['id_cpt_2_sens']=lgn_proche_attr_sup.apply(lambda x : calcul_tmjo_2sens_perm(x['mjo_val'],x['rgraph_dbl'],
                                                                                          x['nb_cpt'], lgn_proche_attr_sup,x['id_grp'],
                                                                                          'mjo_val','id_grp',x['ident_x'],'ident_x')[1],axis=1)

In [70]:
#pour les lignes de cat rhv 1 ou 2 ou 3 : basculer ce trafic vers l'ensemble des lignes du même idtronc issu du cat_rhv_123
mjo_tronc_cat123=lgn_proche_attr_sup.loc[lgn_proche_attr_sup['cat_rhv'].isin(['1','2','3','61','62','63'])][['ident_lgn','tmjo_2_sens','id_cpt_2_sens']].merge(
    gdf_rhv_groupe_123[['ident','idtronc']].rename(columns={'ident':'ident_lgn'}),how='left')#[['idtronc','tmjo_2_sens']]
mjo_tronc_cat123=mjo_tronc_cat123.drop_duplicates(['idtronc','tmjo_2_sens'])

#il rest des données dupliquée, notamment car certains troncons supportent plueiusr points de comptages. dans ce cas on ne garde que la valeur max (vérifié)
traf_max=mjo_tronc_cat123.loc[mjo_tronc_cat123.duplicated('idtronc',keep=False)].groupby('idtronc').tmjo_2_sens.max().reset_index().merge(
mjo_tronc_cat123, on='idtronc')
traf_max=traf_max.loc[traf_max['tmjo_2_sens_x']==traf_max['tmjo_2_sens_y']].drop('tmjo_2_sens_y',axis=1).rename(columns={'tmjo_2_sens_x':'tmjo_2_sens'}).set_index('idtronc').copy()
mjo_tronc_cat123.set_index('idtronc',inplace=True)
mjo_tronc_cat123.update(traf_max.reset_index().set_index('idtronc'))
mjo_tronc_cat123.reset_index(inplace=True)
mjo_tronc_cat123_v2=mjo_tronc_cat123.drop_duplicates(['idtronc','tmjo_2_sens'])
#jointure entre l'idtronc issu des cat 1,2,3 et les lignes qui y sont affectées puis jointure avec la df de base des lignes
gdf_rhv_cpt_perm_123=gdf_rhv_groupe_123.merge(mjo_tronc_cat123_v2, on='idtronc', how='left')
gdf_rhv_cpt_perm_123.loc[~gdf_rhv_cpt_perm_123.tmjo_2_sens.isna(),'type_cpt']='permanent'
gdf_rhv_cpt_perm_123.drop('ident_lgn',axis=1, inplace=True)
#export
gdf_rhv_cpt_perm_123[['id_x', 'ident', 'domanial', 'groupe', 'cat_dig', 'cat_rhv', 'passage',
       'rggraph_nd', 'rggraph_na', 'rgraph_dbl', 'numero', 'cdate', 'mdate',
       'id_ign', 'nature', 'sens', 'codevoie_d', 'importance', 'id_y',
       'idtronc', 'geometry','tmjo_2_sens', 'type_cpt']].to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_trafic_123_v0.shp')

> ### ensuite les comptages ponctuels

In [71]:
#à partir de affect final, chercher les points qui supporte pas déjà un tafic d'un comptage permanent (attention, on se base sur les troncon elementaires toute
#catégorie rhv)
cpt_ponct_ok=cpt_pct_l93.merge(affect_finale.loc[~affect_finale.idtronc.isin(lgn_proche_perm.idtronc.tolist())][['ident','idtronc']], on='ident')

In [72]:
#trouver la ligne la plus proche
lgn_proche_ponct=cpt_ponct_ok.merge(plus_proche_voisin(cpt_ponct_ok,gdf_rhv_groupe,10,'ident','ident'),left_on='ident', right_on='ident_left',how='left').merge(
    gdf_rhv_groupe[['ident','cat_rhv','rgraph_dbl','idtronc']], left_on='ident_right', right_on='ident', how='left').rename(
    columns={'ident_right':'ident_lgn','idtronc_x':'idtronc_tt_rhv','ident_x':'ident'}).drop(['ident_left','ident_y','idtronc_y'],axis=1)

c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [73]:
#calculer le trafic total par point de comptage permanent
def calcul_tmjo_2sens_ponct(tmjo, rgraph_dbl, nb_cpt,sens_uniq, df,id_grp,nom_attr_trafic,nom_attr_id_grp,id_cpt,nom_attr_id_cpt) :
    """calculer le tmja 2 sens en fonction du nb de compteur et du sens unique ou non"""
    if nb_cpt==2 : 
        if (df.loc[df[nom_attr_id_grp]==id_grp].sens_unique==True).all() : #si sur unmm idtronc les 2 copt sont en sens uniq, on garde le max
            return (df.loc[df[nom_attr_id_grp]==id_grp][nom_attr_trafic].max(),df.loc[(df[nom_attr_id_grp]==id_grp) & (df[nom_attr_trafic]==
                        df.loc[df[nom_attr_id_grp]==id_grp][nom_attr_trafic].max())][nom_attr_id_cpt].values[0],)
        else :
            return df.loc[df[nom_attr_id_grp]==id_grp][nom_attr_trafic].sum(), df.loc[df[nom_attr_id_grp]==id_grp].groupby(nom_attr_id_grp)[nom_attr_id_cpt].agg(
                lambda x : tuple(x)).values[0]
    else : 
        if rgraph_dbl==1 : 
            return tmjo*2, (id_cpt,)
        else : 
            return tmjo, (id_cpt,)
        
#calcul du tmjo_2_sens
lgn_proche_ponct_attr_sup=lgn_proche_ponct.merge(lgn_proche_ponct.groupby('idtronc_tt_rhv')['ident'].nunique(), on='idtronc_tt_rhv').rename(
    columns={'ident_y':'nb_cpt','ident_x':'ident'})

In [74]:
lgn_proche_ponct_attr_sup['tmjo_2_sens']=lgn_proche_ponct_attr_sup.apply(lambda x : 
                        calcul_tmjo_2sens_ponct(x['tmjo_tv'],x['rgraph_dbl'],x['nb_cpt'],x['sens_unique'], lgn_proche_ponct_attr_sup,
                                          x['idtronc_tt_rhv'],'tmjo_tv','idtronc_tt_rhv',x['ident'],'ident')[0],axis=1)
lgn_proche_ponct_attr_sup['id_cpt_2_sens']=lgn_proche_ponct_attr_sup.apply(lambda x : 
                        calcul_tmjo_2sens_ponct(x['tmjo_tv'],x['rgraph_dbl'],x['nb_cpt'],x['sens_unique'], lgn_proche_ponct_attr_sup,
                                          x['idtronc_tt_rhv'],'tmjo_tv','idtronc_tt_rhv',x['ident'],'ident')[1],axis=1)

In [ ]:
#verif doublon de compteur --si besoin
lgn_proche_ponct_attr_sup.loc[lgn_proche_ponct_attr_sup.duplicated('ident')]

In [75]:
# 2 cas possibles : soit le comptage ponctuel est sur un troncon dejà concerné issu des troncon elementaire categorie 1,2,3 ou non

#séparer les points restant dans les 2 catégories : 
ponct_sur_perm_123=lgn_proche_ponct_attr_sup.loc[lgn_proche_ponct_attr_sup['ident_lgn'].isin(
    gdf_rhv_cpt_perm_123.loc[gdf_rhv_cpt_perm_123['type_cpt']=='permanent'].ident.tolist())].copy()
ponct_libre_123=lgn_proche_ponct_attr_sup.loc[~lgn_proche_ponct_attr_sup['ident_lgn'].isin(
    gdf_rhv_cpt_perm_123.loc[gdf_rhv_cpt_perm_123['type_cpt']=='permanent'].ident.tolist())].copy()

In [76]:
#rappatriement des compteurs sur les idtronc des cat 1,2,3 : il y a des doublons car les troncons sont long, donc on separe, et on va traiter les ponct_libr sans doublons
ponct_libre_123_tot=gdf_rhv_groupe_123[['ident','idtronc']].merge(ponct_libre_123[['ident_lgn','tmjo_2_sens','id_cpt_2_sens']].rename(columns={'ident_lgn':'ident'}), 
    on='ident')[['idtronc','tmjo_2_sens','id_cpt_2_sens']].drop_duplicates(['idtronc','tmjo_2_sens','id_cpt_2_sens'])
list_pct_libre_dbl=ponct_libre_123_tot.loc[ponct_libre_123_tot.duplicated('idtronc',keep=False)].sort_values('idtronc').idtronc.unique()
#affectation des ponctuels libre sans doublons
pct_libre_123_ss_dbl=ponct_libre_123_tot.loc[~ponct_libre_123_tot['idtronc'].isin(list_pct_libre_dbl)].copy()
pct_libre_123_ss_dbl['type_cpt']='ponctuel'
gdf_rhv_cpt_perm_123.set_index('idtronc', inplace=True)
gdf_rhv_cpt_perm_123.update(pct_libre_123_ss_dbl.set_index('idtronc'))
#gdf_rhv_groupe_123_pctLibreSsDbl=gdf_rhv_groupe_123.merge(pct_libre_123_ss_dbl, on='idtronc')
#gdf_rhv_groupe_123_pctLibreSsDbl.loc[~gdf_rhv_groupe_123_pctLibreSsDbl.tmjo_2_sens.isna(),'type_cpt']='ponctuel'
#gdf_rhv_groupe_123_pctLibreSsDbl[['id_x', 'ident', 'domanial', 'groupe', 'cat_dig', 'cat_rhv', 'passage',
       #'rggraph_nd', 'rggraph_na', 'rgraph_dbl', 'numero', 'cdate', 'mdate',
       #'id_ign', 'nature', 'sens', 'codevoie_d', 'importance', 'id_y',
       #'idtronc', 'geometry','tmjo_2_sens']].to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_trafic_123_pctSsDblv0.shp')

In [77]:
#affectation des ponctuel libre avec doublons
#l'idee c'est pour un idtronc des cat 1,2,3 on a la liste des ident, on affecte sur les ident des idtronc elementaire 
#sur les lignes, puis on propage sur les tronc de la cat 1,2,3

pct_libre_123_avec_dbl=ponct_libre_123_tot.loc[ponct_libre_123_tot['idtronc'].isin(list_pct_libre_dbl)].copy()
gdf_rhv_cpt_perm_123=gdf_rhv_cpt_perm_123.reset_index().set_index('ident')

In [ ]:
for idtronc_test in sorted(pct_libre_123_avec_dbl.idtronc.unique()) : 
    print(idtronc_test)
    #récupérer l'idtronc toute cat_rhv des compteurs avec doublons et faire la jointure du trafic sur les lignes concernees
    pct_libre_123_avec_dbl_decompose=lgn_proche_ponct_attr_sup.loc[lgn_proche_ponct_attr_sup['ident'].isin(
        [a for b in pct_libre_123_avec_dbl.loc[pct_libre_123_avec_dbl['idtronc']==idtronc_test].
         id_cpt_2_sens.tolist() for a in b])][['idtronc_tt_rhv','tmjo_2_sens','id_cpt_2_sens']].copy()

    #initialisation des paramètres : 
    ligne_ok=gdf_rhv_groupe.merge(pct_libre_123_avec_dbl_decompose.rename(columns={'idtronc_tt_rhv':'idtronc'}), on='idtronc').merge(
        graph_filaire_123[['ident','source','target']], on='ident').drop(['source_x', 'target_x'],axis=1).rename(
        columns={'source_y':'source', 'target_y':'target'})#lignes avec trafic issues des troncons elemntaires et on garde les ousrces et target liées uniquement aux cat 1,2,3
    lgn_id_tronc_123_vide=gdf_rhv_groupe_123.loc[(gdf_rhv_groupe_123['idtronc']==idtronc_test) & 
                                                 (~gdf_rhv_groupe_123.ident.isin(ligne_ok.ident.tolist()))] #l'ensemble des lignes a renseigner
    while ~lgn_id_tronc_123_vide.empty :
        list_noeud_trafic=ligne_ok.source.tolist()+ligne_ok.target.tolist() #les noeuds avec du trafic
        list_noeud_trafic_null=lgn_id_tronc_123_vide.source.tolist()+lgn_id_tronc_123_vide.target.tolist() #l'inverse
        lgn_a_renseigner=lgn_id_tronc_123_vide.loc[((lgn_id_tronc_123_vide.source.isin(list_noeud_trafic)) & (lgn_id_tronc_123_vide.target.isin(list_noeud_trafic_null))) | 
                ((lgn_id_tronc_123_vide.source.isin(list_noeud_trafic_null)) & (lgn_id_tronc_123_vide.target.isin(list_noeud_trafic)))].copy()
        if lgn_a_renseigner.empty : break #si pas de ligne a renseigner on sort
        ligne_ok['noeud_partage']=ligne_ok.apply(lambda x : x['source'] if x['source'] in list_noeud_trafic_null else x['target'], axis=1) #pour faire une jointure entre les lignes a trafic et les autres
        lgn_a_renseigner['noeud_partage']=lgn_a_renseigner.apply(lambda x : x['source'] if x['source'] in list_noeud_trafic else x['target'], axis=1)
        lgn_a_renseigner=lgn_a_renseigner.merge(ligne_ok[['noeud_partage','tmjo_2_sens','id_cpt_2_sens']], on='noeud_partage') #recupere le tmj
        ligne_ok=pd.concat([ligne_ok,lgn_a_renseigner], axis=0, sort=False)#on ajoute les lignes avec du trafic aux autres
        lgn_id_tronc_123_vide=lgn_id_tronc_123_vide.loc[~lgn_id_tronc_123_vide.ident.isin(ligne_ok.ident.tolist())].copy() #on met à jour pour la boucle
    ligne_ok.drop_duplicates('ident',inplace=True)# les ronds points crees des doublons
    ligne_ok['type_cpt']='ponctuel'
    #mettre à jour la donnees de base
    gdf_rhv_cpt_perm_123.update(ligne_ok[['ident','tmjo_2_sens','id_cpt_2_sens','type_cpt']].set_index('ident'))
#export
#gdf_rhv_groupe_123_pctLibreAvecDbl.reset_index()[['id_x', 'ident', 'domanial', 'groupe', 'cat_dig', 'cat_rhv', 'passage',
       #'rggraph_nd', 'rggraph_na', 'rgraph_dbl', 'numero', 'cdate', 'mdate',
       #'id_ign', 'nature', 'sens', 'codevoie_d', 'importance', 'id_y',
       #'idtronc', 'geometry','tmjo_2_sens']].to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_trafic_123_pctAvecDblv0.shp')

In [ ]:
#affectation des ponctuels sur permanents : de la mm façon que les ponctuels en doublons

#trouver la liste des idtronc cat_rhv = 1,2,3 avec ponct sur perm
idtronc123_pct_sur_perm=gdf_rhv_groupe_123[['ident','idtronc']].merge(ponct_sur_perm_123[['ident_lgn','tmjo_2_sens','id_cpt_2_sens']].rename(columns={'ident_lgn':'ident'}), 
    on='ident')[['idtronc','tmjo_2_sens','id_cpt_2_sens']].drop_duplicates()

for idtronc_test in idtronc123_pct_sur_perm.idtronc.unique() : 
    print(idtronc_test)
    #trouver les id tronc elementaire des comptages ponct et perm concerne
    ligne_ok= gdf_rhv_groupe.merge(pd.concat([lgn_proche_ponct_attr_sup.loc[lgn_proche_ponct_attr_sup['ident'].isin(
        [a for b in idtronc123_pct_sur_perm.loc[idtronc123_pct_sur_perm['idtronc']==idtronc_test].id_cpt_2_sens.tolist()
         for a in b])][['idtronc_tt_rhv','tmjo_2_sens','id_cpt_2_sens']].drop_duplicates(['idtronc_tt_rhv','tmjo_2_sens','id_cpt_2_sens']),
    gdf_rhv_groupe_123.loc[gdf_rhv_groupe_123['idtronc']==idtronc_test].merge(
        lgn_proche_attr_sup, left_on='ident', right_on='ident_lgn')[['idtronc_y','tmjo_2_sens','id_cpt_2_sens']].rename(
        columns={'idtronc_y' : 'idtronc_tt_rhv'}).drop_duplicates()],axis=0, sort=False), left_on='idtronc', right_on='idtronc_tt_rhv')

    lgn_id_tronc_123_vide=gdf_rhv_groupe_123.loc[(gdf_rhv_groupe_123['idtronc']==idtronc_test) & 
                                                     (~gdf_rhv_groupe_123.ident.isin(ligne_ok.ident.tolist()))]

    while ~lgn_id_tronc_123_vide.empty :
        list_noeud_trafic=ligne_ok.source.tolist()+ligne_ok.target.tolist() #les noeuds avec du trafic
        list_noeud_trafic_null=lgn_id_tronc_123_vide.source.tolist()+lgn_id_tronc_123_vide.target.tolist() #l'inverse
        lgn_a_renseigner=lgn_id_tronc_123_vide.loc[((lgn_id_tronc_123_vide.source.isin(list_noeud_trafic)) & (lgn_id_tronc_123_vide.target.isin(list_noeud_trafic_null))) | 
                ((lgn_id_tronc_123_vide.source.isin(list_noeud_trafic_null)) & (lgn_id_tronc_123_vide.target.isin(list_noeud_trafic)))].copy()
        if lgn_a_renseigner.empty : break #si pas de ligne a renseigner on sort
        ligne_ok['noeud_partage']=ligne_ok.apply(lambda x : x['source'] if x['source'] in list_noeud_trafic_null else x['target'], axis=1) #pour faire une jointure entre les lignes a trafic et les autres
        lgn_a_renseigner['noeud_partage']=lgn_a_renseigner.apply(lambda x : x['source'] if x['source'] in list_noeud_trafic else x['target'], axis=1)
        lgn_a_renseigner=lgn_a_renseigner.merge(ligne_ok[['noeud_partage','tmjo_2_sens','id_cpt_2_sens']], on='noeud_partage') #recupere le tmj
        ligne_ok=pd.concat([ligne_ok,lgn_a_renseigner], axis=0, sort=False)#on ajoute les lignes avec du trafic aux autres
        lgn_id_tronc_123_vide=lgn_id_tronc_123_vide.loc[~lgn_id_tronc_123_vide.ident.isin(ligne_ok.ident.tolist())].copy() #on met à jour pour la boucle
    ligne_ok.drop_duplicates('ident',inplace=True)# les ronds points crees des doublons
    ligne_ok['type_cpt']='ponctuel'
    #mettre à jour la donnees de base
    gdf_rhv_cpt_perm_123.update(ligne_ok[['ident','tmjo_2_sens','id_cpt_2_sens','type_cpt']].set_index('ident'))

#gdf_rhv_groupe_123_pctLibreAvecDbl.reset_index()[['id_x', 'ident', 'domanial', 'groupe', 'cat_dig', 'cat_rhv', 'passage',
       #'rggraph_nd', 'rggraph_na', 'rgraph_dbl', 'numero', 'cdate', 'mdate',
       #'id_ign', 'nature', 'sens', 'codevoie_d', 'importance', 'id_y',
       #'idtronc', 'geometry','tmjo_2_sens']].to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_trafic_123_pctAvecDblv01.shp')

In [80]:
gdf_rhv_cpt_perm_123['id_cpt_2_sens'].fillna('NC', inplace=True)
gdf_rhv_cpt_perm_123['id_cpt_2_sens']=gdf_rhv_cpt_perm_123.apply(lambda x : ', '.join([str(a) for a in x['id_cpt_2_sens']] 
                if not isinstance(x['id_cpt_2_sens'],np.int64) else str(x['id_cpt_2_sens']) ), axis=1)
gdf_rhv_cpt_perm_123.reset_index()[['id_x', 'ident', 'domanial', 'groupe', 'cat_dig', 'cat_rhv', 'passage',
       'rggraph_nd', 'rggraph_na', 'rgraph_dbl', 'numero', 'cdate', 'mdate',
       'id_ign', 'nature', 'sens', 'codevoie_d', 'importance', 'id_y',
       'idtronc', 'geometry','tmjo_2_sens','type_cpt', 'id_cpt_2_sens']].to_file(
    r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_trafic_123_v1.shp')

# On peut essayer de trouver les lignes qui intersectent d'autres avec lignes avec du trafci pour les completer

In [81]:
#trouver les noeuds du graph pour lesquels les lignes qui arrivent

#simplifier la topologie en affectant une seule valuer de noeud aux voie arrivant sur un rond point
#trouver les noeud faisant partie d'un rond point 

df=import_donnes_base('local_otv','linearisation_bm', 'graph_rhv_123','graph_rhv_123_vertices_pgr')
df2_chaussees=df.loc[df.nature.isin(['Autoroute', 'Quasi-autoroute', 'Route à 2 chaussées'])]
df_avec_rd_pt,carac_rd_pt,lign_entrant_rdpt=identifier_rd_pt(df)

lgn_rdpt=df_avec_rd_pt.loc[~df_avec_rd_pt.id_rdpt.isna()].copy()
dico_noeud=rt.creer_dico_noeud_rdpt(lgn_rdpt)

c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [120]:
# àpartir du dico on va remplacer les valeurs de source ou target par la valeur de clé du dico. 
gdf_rhv_rdpt_simple=gdf_rhv_cpt_perm_123.loc[~gdf_rhv_cpt_perm_123.index.isin(lgn_rdpt.ident.to_list())].copy()
#remplacement des osurces et targets : 
simplifier_noeud_rdpt(gdf_rhv_rdpt_simple, dico_noeud)
#puis recalculer le count du nb de ligne par noeud (en otant les lignes qui font les rdpoints)
cnt_maj=maj_graph_rdpt(gdf_rhv_rdpt_simple)

#trouver les noeuds fv concernes par des lignes à mettre à jour
noeud_grp,df_noeud_tot=noeud_fv_ligne_ss_trafic(gdf_rhv_rdpt_simple,'max 1 NaN')

#se limiter aux noeuds présentant 3 lignes (pour rappel, seul une ligne présente un trafic inconnu)
liste_noeud=noeud_grp.loc[(noeud_grp['estimable']) & 
             (noeud_grp.apply(lambda x : len(x['tmjo_2_sens'])==3, axis=1))]

In [96]:
#4.calculer le trafic sur la voie manquante (A FAIRE)

def calcul_trafic_manquant(num_noeud, df, df_tot, nom_idtroncon, graph) : 
    """
    detreminer du trafic sur unnoeud a 3 voies avec 2 trafic connu
    in : 
        num_noeud : identifiant du noeud
        df : df des lignes liées au noeud
        df_tot : df globale du reseau routier sans simplification des rd points. doit contenir un attribut de groupement par troncon
        nom_idtroncon : string : nom de l'attribut de groupement par troncon
    """
    
        
    def nb_sens(rgraph_dbl,nb_noeud_unique,nb_noeud,nb_lgn_tch_noeud_unique,nb_ligne) : 
        """
        savoir si un troncon est en double sens ou non
        """
        if rgraph_dbl==0 :
            if nb_noeud_unique > 2 :
                if nb_lgn_tch_noeud_unique > 2 : 
                    if nb_ligne==nb_noeud-1 : 
                        return 0
                    elif nb_ligne==nb_noeud-2 : 
                        return 1
                elif nb_lgn_tch_noeud_unique == 2 :
                    return 1
            else : return 0
        else: return 1
        
    df_calcul=df.copy()
    
    #verification de l'attribut rggraph_dbl
    #ça peut etre vraiment un voie à sens unique et chaussées unique, ou alors c'est un cas genre les boulevard, qui est note sens unique mais est en realite une voie double sens
    #on va donc regarder le nombre de noeud unique pour savoir si le troncon contient des lignes parralleles ou non. si c le cas alors c un troncon double sens
    df_calcul['list_noeud_unique']=df_calcul.apply(lambda x : rt.Troncon(df_tot, x['idtronc']).noeuds_uniques, axis=1) #liste des noeuds en fin de troncon
    df_calcul['nb_noeud_unique']=df_calcul.apply(lambda x : rt.Troncon(df_tot, x['idtronc']).nb_noeuds_uniques, axis=1) #nb de noeud en fin de troncon
    df_calcul['nb_noeud']=df_calcul.apply(lambda x : rt.Troncon(df_tot, x['idtronc']).nb_noeuds, axis=1) #nb de noeud en fin de troncon
    df_calcul['nb_lgn_tch_noeud_unique']=df_calcul.apply(lambda x : rt.Troncon(df_tot, x['idtronc']).nb_lgn_tch_noeud_unique, axis=1) 
    df_calcul['nb_lgn']=df_calcul.apply(lambda x : rt.Troncon(df_tot, x['idtronc']).nb_lgn, axis=1)
    #ajout d'un attribut supplémentaire
    df_calcul['rgraph_dbl_2']=df_calcul.apply(lambda x : nb_sens(x['rgraph_dbl'],x['nb_noeud_unique'],x['nb_noeud'],x['nb_lgn_tch_noeud_unique'],x['nb_lgn']), axis=1)
    
    
    #verification que le troncon a mettre à jour ne croise bien que 2 autres troncons pour les voies à 2 chaussées 
    if (df_calcul.loc[df_calcul['maj']].rgraph_dbl==0).all() and (df_calcul.loc[df_calcul['maj']].rgraph_dbl_2==1).all() : 
        #trouver le troncon à mettre à jour :
        troncon=rt.Troncon(gdf_rhv_rdpt_simple,df_noeud.loc[df_noeud['maj']].idtronc.values[0])
        #sutilisation de la cle de correspondance entre les neouds
        corresp_noeud_uniq=troncon.groupe_noeud_route_2_chaussees(graph,100)
        noeud_parrallele=corresp_noeud_uniq.loc[corresp_noeud_uniq['id_left']==num_noeud].id_right.values[0]
        #trouver les troncon qui intersectent ce troncon sur le debut ou la fin et qui n'ont pas été fléchés au début
        df_troncons_sup=df_tot.loc[((df_tot['source']==noeud_parrallele) | (df_tot['target']==noeud_parrallele)) & 
                               (~df_tot.idtronc.isin(df_calcul.idtronc.tolist()))]
        if not df_troncons_sup.empty : 
            raise PasDeTraficError(troncon.id)
    
    
    #print(df_calcul)
    df_calcul_trafic_exist=df_calcul.loc[df_calcul['tmjo_2_sens']!=-99]
    df_calcul_trafic_null=df_calcul.loc[df_calcul['tmjo_2_sens']==-99]
    
    #calcul selon les cas de sens unique ou non
    if (df_calcul_trafic_exist.rgraph_dbl_2==0).all() : 
        if len(df_calcul_trafic_exist.type_noeud.unique())==1 : 
            return df_calcul_trafic_exist.tmjo_2_sens.sum()
        else : 
            if (df_calcul_trafic_null.type_noeud=='a').all() :
                return (df_calcul_trafic_exist.loc[df_calcul_trafic_exist['type_noeud']=='d'].tmjo_2_sens.values[0] - 
                        df_calcul_trafic_exist.loc[df_calcul_trafic_exist['type_noeud']=='a'].tmjo_2_sens.values[0])
            else :
                return (df_calcul_trafic_exist.loc[df_calcul_trafic_exist['type_noeud']=='a'].tmjo_2_sens.values[0] - 
                 df_calcul_trafic_exist.loc[df_calcul_trafic_exist['type_noeud']=='d'].tmjo_2_sens.values[0])
    elif (df_calcul_trafic_exist.rgraph_dbl_2!=0).all() :
        return df_calcul_trafic_exist.tmjo_2_sens.max()-df_calcul_trafic_exist.tmjo_2_sens.min()
    else : 
        if (df_calcul_trafic_null.rgraph_dbl_2==1).all() : 
            return df_calcul_trafic_exist.loc[df_calcul_trafic_exist['rgraph_dbl']==1].tmjo_2_sens.values[0]

class PasDeTraficError(Exception):
    """
    Exception levée si la ligne n'est pas affectée à du trafic
    """
    def __init__(self, idtroncon):
        Exception.__init__(self,f'pas de trafic sur le troncon : {idtroncon}')
        self.erreur_type='PasDeTraficError'

In [109]:
gdf_rhv_extrapol_1=gdf_rhv_cpt_perm_123.reset_index().set_index('idtronc') #initialiser pour resultats    
list_noeud_pb={}
#for noeud in liste_noeud.noeud.tolist() :
noeud=8235
print(noeud)
df_temp=gdf_rhv_rdpt_simple.loc[(gdf_rhv_rdpt_simple['source']==noeud) | (gdf_rhv_rdpt_simple['target']==noeud)].copy()
df_temp['type_noeud']=df_temp.apply(lambda x : 'd' if x['source']==noeud else 'a', axis=1 )
df_noeud=df_noeud_tot.loc[df_noeud_tot['noeud']==noeud].merge(df_temp[['id_ign','type_noeud', 'idtronc']], on='id_ign')
df_noeud['maj']=df_noeud.apply(lambda x : True if x['tmjo_2_sens']==-99 else False, axis=1)
try : 
    df_noeud.loc[df_noeud['tmjo_2_sens']==-99,'tmjo_2_sens']=df_noeud.apply(lambda x : calcul_trafic_manquant(noeud,df_noeud,gdf_rhv_cpt_perm_123,'idtronc',graph_filaire_123_vertex), axis=1)
except PasDeTraficError :
    list_noeud_pb[noeud]='PasDeTraficError'
except IndexError : 
    list_noeud_pb[noeud]='IndexError'
if df_noeud.tmjo_2_sens.isnull().any() : 
    list_noeud_pb[noeud]='autre Pb trafic'
gdf_rhv_extrapol_1.update(df_noeud.loc[df_noeud['maj']][['idtronc','tmjo_2_sens']].set_index('idtronc'))

8235


In [110]:
df_noeud

,id_ign,noeud,tmjo_2_sens,cat_rhv,rgraph_dbl,type_noeud,idtronc,maj
0,TRONROUT41444,8235,1833.0,3,0.0,d,6013.0,False
1,TRONROUT41296,8235,7402.0,3,1.0,d,4760.0,False
2,TRONROUT7036,8235,7402.0,3,1.0,a,7883.0,True


In [117]:
list_noeud_pb

{}

In [116]:
gdf_rhv_extrapol_1.reset_index()[['id_x', 'ident', 'domanial', 'groupe', 'cat_dig', 'cat_rhv', 'passage',
       'rggraph_nd', 'rggraph_na', 'rgraph_dbl', 'numero', 'cdate', 'mdate',
       'id_ign', 'nature', 'sens', 'codevoie_d', 'importance', 'id_y',
       'idtronc', 'geometry','tmjo_2_sens','type_cpt', 'id_cpt_2_sens','source','target']].to_file(
    r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_trafic_extrapol.shp')

## 2. MMM
En premier lieu il faut mettre à jour les attributs en renomant de façon explicite puis en faisant la somme des 2 sens de circulation pour les voies doucble sens

In [22]:
#importer les données et convertir les noms de champs
fichier_src=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\MMM\2017_Matin_PR\2017_HPM_link.SHP')
#définir les colonnes définitives (PLUS TARD ON FERA UN FILTRE EN AMONT)
liste_col_def=['NO', 'FROMNODENO', 'TONODENO', 'TYPENO', 'TSYSSET', 'LENGTH', 'NUMLANES', 'CAPPRT', 'V0_TV', 'VOLVEHPR~1', 
'Q_HC_PL', 'Q_HC_TV', 'Q_HC_VL', 'Q_HPM_PL', 'Q_HPM_TV', 'Q_HPM_VL', 'Q_HPS_PL', 'Q_HPS_TV', 'Q_HPS_VL', 'Q_Jour_PL', 'Q_Jour_TV',
'Q_Jour_VL','coef_HC_PL','coef_HC_TV','coef_HC_VL','Ocup_VL_PL','V0_PL','V0_VL','VCharg_PL','VCharg_VL','VOLPCUP~22',
'SHAREHGV', 'VOLVEH_~23', 'VOLVEH_~24', 'VOLCAPR~25', 'VEHHOUR~26', 'SPEEDLIMIT',
'R_NO', 'R_FROMNODENO', 'R_TONODENO', 'R_TYPENO', 'R_TSYSSET','R_LENGTH', 'R_NUMLANES', 'R_CAPPRT', 'r_V0_TV', 'R_VOLVEHPR~1', 
'R_Q_HC_PL', 'R_Q_HC_TV', 'R_Q_HC_VL', 'R_Q_HPM_PL', 'R_Q_HPM_TV', 'R_Q_HPM_VL', 
'R_Q_HPS_PL', 'R_Q_HPS_TV', 'R_Q_HPS_VL', 'R_Q_Jour_PL', 'R_Q_Jour_TV', 'R_Q_Jour_VL', 'R_coef_HC_PL', 'R_coef_HC_TV', 'R_coef_HC_VL', 'R_Ocup_VL_PL', 'R_V0_PL', 
'R_V0_VL', 'R_VCharg_PL', 'R_VCharg_VL', 'R_VOLPCUP~22', 'R_SHAREHGV', 'R_VOLVEH_~23', 'R_VOLVEH_~24', 'R_VOLCAPR~25', 'R_VEHHOUR~26', 'R_SPEEDLIMIT']
#creer un dico de renomage et renommer
dico_renomage={a: b for a, b in zip(fichier_src.columns,liste_col_def)}
fichier_src.rename(columns=dico_renomage, inplace=True)
#ne conserver dans un premier temps que les attributs relatifs au trafic sur la journée
fichier_src_simpl=fichier_src[['geometry','NO', 'FROMNODENO', 'TONODENO', 'TYPENO', 'TSYSSET', 'LENGTH', 'NUMLANES', 'CAPPRT', 'V0_TV', 
             'VOLVEHPR~1']+[a for a in fichier_src.columns if 'Jour' in a ]].copy()
#caculer le tmja_TV
fichier_src_simpl['tmja_tv']=fichier_src_simpl.Q_Jour_TV+fichier_src_simpl.R_Q_Jour_TV

In [14]:
fichier_src_simpl.head(5)

,NO,FROMNODENO,TONODENO,TYPENO,TSYSSET,LENGTH,NUMLANES,CAPPRT,V0_TV,VOLVEHPR~1,Q_Jour_PL,Q_Jour_TV,Q_Jour_VL,R_Q_Jour_PL,R_Q_Jour_TV,R_Q_Jour_VL,tmja_tv
0,2000062,2000054,5868961,82,"A,B,BHNS,P,R,V",0.114km,1,600,40km/h,314,221.68,4422.38,4200.70,294.99,5945.08,5650.08,10367.46
1,2000063,2000054,6389027,82,"A,B,BHNS,P,R,V",0.099km,1,600,40km/h,467,294.99,5945.08,5650.08,221.68,4422.38,4200.70,10367.46
2,2000064,3527,2000055,77,"A,B,BHNS,P,R,V",0.160km,1,750,40km/h,269,582.93,9835.65,9190.25,424.10,6218.11,5755.06,16053.76
3,2000065,2000055,4852848,77,"A,B,BHNS,P,R,V",0.037km,1,750,40km/h,269,582.93,9835.65,9190.25,424.10,6218.11,5755.06,16053.76
4,2000068,5743212,20000010,82,"A,B,BHNS,P,R,V",0.074km,1,600,40km/h,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [ ]:
fichier_src_simpl.to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\MMM_simplifie.shp')

In [26]:
#creer graph et importer 
creer_graph(fichier_src_simpl, 'local_otv',id_name='NO', schema='public', table='mmm_graph', table_vertex='mmm_graph_vertices_pgr')

In [472]:
#importer le graph
with ct.ConnexionBdd('local_otv') as c :
    graph_mmm_filaire = gp.read_postgis('select * from linearisation_bm.mmm_graph', c.connexionPsy)
    graph_mmm_filaire_vertex = gp.read_postgis('select * from linearisation_bm.mmm_graph_123_vertices_pgr', c.connexionPsy)

In [475]:
#importer le fichier simplifie
mmm_simple=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\MMM_simplifie.shp')

## définir des relations entre les voies, à chaque point qui intersecte plus de 2 voie

In [473]:
#les noued s sup à 2 voies
noeud_sup2Voies=graph_mmm_filaire_vertex.loc[graph_mmm_filaire_vertex['cnt']>2].copy()

In [702]:
#trouver la correspondance entre les noeuds MMM et FV
#depuis le MMM on cherche le noeud FV le plus proche et inversement,puis on fusionne
appariement_noeud_mmm_fv=rt.appariement_noeud_mmm_fv(noeud_sup2Voies, graph_filaire_123_vertex, 30)

In [653]:
#trouver la correspondanceentre les lignes
#importer les données de correspondance
with ct.ConnexionBdd('local_otv') as c :
    cle_mmm_rhv = gp.read_postgis('select * from linearisation_bm.appariement_etape1_2', c.connexionPsy)

In [794]:
#trouver les noeuds fv concernes par des lignes à mettre à jour
noeud_grp=noeud_fv_ligne_ss_trafic(gdf_rhv_cpt_perm_123,'min 1 ok')[0]
#ne conserver que le noeuds relatif à une certaine catégorie
liste_noeud_a_traiter=noeud_grp.loc[(noeud_grp['estimable']) & (noeud_grp.apply(lambda x : '1' in x['cat_rhv'],axis=1))].noeud.tolist()

In [791]:
gdf_rhv_extrapol=gdf_rhv_cpt_perm_123.reset_index().set_index('idtronc')
#pour un noeud
noeud_fv=783
#voies rhv concernees par le noeud
matrice_voie_rhv=rt.corresp_noeud_rhv(gdf_rhv_cpt_perm_123,noeud_fv)
#jointure via cle de passage avec voies du MMM pour la partie entrante. Si une des voies a pas de jointure  : pb ; si plusiuers voies MMM à joindre : pb
joint_fv_mmm_e1=matrice_voie_rhv.merge(cle_mmm_rhv[['NO','ident']], left_on='ident_x', right_on='ident').drop_duplicates().rename(
        columns={'NO':'NO_x'}).drop('ident', axis=1)
if joint_fv_mmm_e1.NO_x.isna().any() : 
    print('pb')
joint_fv_mmm_e2=joint_fv_mmm_e1.merge(cle_mmm_rhv[['NO','ident']], left_on='ident_y', right_on='ident').drop_duplicates().rename(
        columns={'NO':'NO_y'}).drop('ident', axis=1)
if joint_fv_mmm_e2.NO_y.isna().any() : 
    print('pb')
#on ne garde que les voies qui ont un des deux tmjo avec un trafic inconnu
trafic_inconnus=joint_fv_mmm_e2.loc[joint_fv_mmm_e2.tmjo_2_sens_x.isna() | joint_fv_mmm_e2.tmjo_2_sens_y.isna()].copy()
#on ne garde que les ligne de catégorie la plus proche
trafic_inconnus['diff_cat']=trafic_inconnus.apply(lambda x : abs((int(x['cat_rhv_x'])-int(x['cat_rhv_y']))), axis=1)
trafic_inconnus_prior_cat=trafic_inconnus.loc[trafic_inconnus['diff_cat']==trafic_inconnus.groupby('ident_x').diff_cat.transform(min)]
#passer les trafic du MMM
traf_mmm=trafic_inconnus_prior_cat.merge(mmm_simple[['NO','tmja_tv']], left_on='NO_x', right_on='NO').drop('NO', axis=1).merge(
    mmm_simple[['NO','tmja_tv']], left_on='NO_y', right_on='NO').drop('NO', axis=1)
traf_mmm['traf_max']=traf_mmm.apply(lambda x:max(x['tmja_tv_x'],x['tmja_tv_y']), axis=1)
#calcul des trafics
trafc_rens=traf_mmm.copy()
trafc_rens['ident_a_rens']=trafc_rens.apply(lambda x : x['ident_x'] if pd.isnull(x['tmjo_2_sens_x'])
                                                  else x['ident_y'], axis=1)
trafc_rens['tmjo_2_sens_extrapol']=trafc_rens.apply(lambda x : 
    int(x['tmja_tv_y']/x['tmja_tv_x']*x['tmjo_2_sens_x']) if pd.isnull(x['tmjo_2_sens_y']) else int(x['tmja_tv_x']/x['tmja_tv_y']*x['tmjo_2_sens_y']),
                                                  axis=1)
#si plusieurs resultas possibles pour unident on garde le max
trafc_fin=trafc_rens.loc[trafc_rens['traf_max']==trafc_rens.groupby('ident_a_rens').traf_max.transform(max)].merge(
    gdf_rhv_cpt_perm_123[['idtronc']], left_on='ident_a_rens',right_index=True)[['ident_a_rens','tmjo_2_sens_extrapol','idtronc']].drop_duplicates()

In [ ]:
#mise à jour du fichier source
gdf_rhv_extrapol.update(trafc_fin.set_index('idtronc').rename(columns={'tmjo_2_sens_extrapol':'tmjo_2_sens'})[['tmjo_2_sens']])

In [792]:
trafc_fin

,ident_a_rens,tmjo_2_sens_extrapol,idtronc
0,2647,11232,7616.0


In [697]:
#ensemble de noeud
gdf_rhv_extrapol=gdf_rhv_cpt_perm_123.reset_index().set_index('idtronc')
for noeud_fv in liste_noeud_a_traiter :
    print(noeud_fv)
    #trouver noeud correspondant
    try :
        noeud_mmm_corrsp=appariement_noeud_mmm_fv.loc[appariement_noeud_mmm_fv['id_fv']==noeud_fv].id_mmm.values[0]
    except IndexError : 
        continue        
    #tableau de correspondance entre les voies du mmm
    matrice_voie_mmm=rt.corresp_noeud_mmm(mmm_simple, noeud_mmm_corrsp)
    ##tableau de correspondance entre les voies du rhv
    matrice_voie_rhv=rt.corresp_noeud_rhv(gdf_rhv_cpt_perm_123,noeud_fv)
    #jointure avec les voies du rhv
    corresp_mm_rhv=matrice_voie_mmm.merge(cle_mmm_rhv[['NO','ident']], left_on='NO_x', right_on='NO').drop_duplicates().rename(
        columns={'ident':'ident_x'}).drop('NO', axis=1).merge(cle_mmm_rhv[['NO','ident']], left_on='NO_y', right_on='NO').drop_duplicates().rename(
        columns={'ident':'ident_y'}).drop('NO', axis=1)
    #on ne garde que les voies d rhv présentent dans le table de correspondance
    corresp_mm_rhv_fin=corresp_mm_rhv.loc[(corresp_mm_rhv.ident_x.isin(matrice_voie_rhv.ident_x.tolist()+matrice_voie_rhv.ident_y.tolist())) & 
                      (corresp_mm_rhv.ident_y.isin(matrice_voie_rhv.ident_x.tolist()+matrice_voie_rhv.ident_y.tolist()))]
    #on joint les trafics
    trafic_inconnus=corresp_mm_rhv_fin.merge(gdf_rhv_cpt_perm_123[['tmjo_2_sens', 'cat_rhv']], left_on='ident_x', right_index=True).merge(
        gdf_rhv_cpt_perm_123[['tmjo_2_sens','cat_rhv']], left_on='ident_y', right_index=True)
    #si les trafics inconnus sont vides on continu, on traitera apres
    if trafic_inconnus.empty : 
        continue
    #fonction de deterination des lignes a remplir
    trafic_a_rens=trafic_inconnus.loc[((trafic_inconnus.tmjo_2_sens_x.isna() & trafic_inconnus.tmjo_2_sens_y.notna()) |
    (trafic_inconnus.tmjo_2_sens_y.isna() & trafic_inconnus.tmjo_2_sens_x.notna()))].copy()
    if trafic_a_rens.empty : 
        continue
    #calcul des trafics
    trafic_a_rens['ident_a_rens']=trafic_a_rens.apply(lambda x : x['ident_x'] if pd.isnull(x['tmjo_2_sens_x'])
                                                      else x['ident_y'], axis=1)
    trafic_a_rens['tmjo_2_sens_extrapol']=trafic_a_rens.apply(lambda x : 
        int(x['tmja_tv_y']/x['tmja_tv_x']*x['tmjo_2_sens_x']) if pd.isnull(x['tmjo_2_sens_y']) else int(x['tmja_tv_x']/x['tmja_tv_y']*x['tmjo_2_sens_y']),
                                                      axis=1)
    #filtre des resultats selon la cat_rhv : on prend la plus proche (a faire, pour le moment on prend celle qui est égale)
    trafic_extrapol=trafic_a_rens.loc[trafic_a_rens['cat_rhv_x']==trafic_a_rens['cat_rhv_y']].copy()
    #lien avec l'idtronc concerne pour affectation
    trafic_extrapol=trafic_extrapol.merge(gdf_rhv_cpt_perm_123[['idtronc']], left_on='ident_a_rens', 
                                          right_index=True)[['ident_a_rens','tmjo_2_sens_extrapol','idtronc']]
    #on ne gargee que le trafic max en cas de doublons
    trafic_extrapol=trafic_extrapol.loc[trafic_extrapol['tmjo_2_sens_extrapol']==trafic_extrapol.groupby('idtronc').tmjo_2_sens_extrapol.transform(max)]
    #mise à jour du fichier source

    gdf_rhv_extrapol.update(trafic_extrapol.set_index('idtronc').rename(columns={'tmjo_2_sens_extrapol':'tmjo_2_sens'})[['tmjo_2_sens']])

In [699]:
gdf_rhv_extrapol.reset_index()[['id_x', 'ident', 'domanial', 'groupe', 'cat_dig', 'cat_rhv', 'passage',
       'rggraph_nd', 'rggraph_na', 'rgraph_dbl', 'numero', 'cdate', 'mdate',
       'id_ign', 'nature', 'sens', 'codevoie_d', 'importance', 'id_y',
       'idtronc', 'geometry','tmjo_2_sens','type_cpt', 'id_cpt_2_sens']].to_file(
    r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_trafic_123_extrapol.shp')

In [695]:
trafic_inconnus

,noeud,NO_x,tmja_tv_x,NO_y,tmja_tv_y,ident_x,ident_y,tmjo_2_sens_x,cat_rhv_x,tmjo_2_sens_y,cat_rhv_y
1,6135635.0,1034980026,5948.72,1034980782,23295.99,8782,28019,7787.0,2,38095.0,1


In [686]:
matrice_voie_rhv

,noeud,ident_x,tmjo_2_sens_x,ident_y,tmjo_2_sens_y
1,845,49936,NaN,2830,5133.0
2,845,49936,NaN,50641,14216.0
5,845,2830,5133.0,50641,14216.0


In [688]:
matrice_voie_mmm

,noeud,NO_x,tmja_tv_x,NO_y,tmja_tv_y
1,4722269.0,57894565,1906.02,57931055,7814.92
2,4722269.0,57894565,1906.02,2145032626,9646.26
5,4722269.0,57931055,7814.92,2145032626,9646.26


In [690]:
corresp_mm_rhv

,noeud,NO_x,tmja_tv_x,NO_y,tmja_tv_y,ident_x,ident_y
0,4722269.0,57894565,1906.02,57931055,7814.92,None,None
1,4722269.0,57894565,1906.02,2145032626,9646.26,None,49936
2,4722269.0,57931055,7814.92,2145032626,9646.26,None,49936


## 2. Filaire CBS
En premier lieu il faut mettre à jour les attributs en renomant de façon explicite puis en faisant la somme des 2 sens de circulation pour les voies doucble sens

In [20]:
filaire_cbs=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\Bdx-Metro\CBS2017\Filaire_global_metropole_DENSIFIER.shp')

In [30]:
filaire_cbs['tmja']=filaire_cbs.apply(lambda x : (float(x['MT'].replace(',','.'))*12)+
                                      (float(x['ME'].replace(',','.'))*4)+(float(x['MN'].replace(',','.'))*8),axis=1)

In [32]:
filaire_cbs.to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\filaire_tmja.shp')

# TESTS

In [1203]:
#trouver les noeuds du troncon concerne
toto=rt.Troncon(gdf_rhv_cpt_perm_123,934)
noeud_uniq=toto.noeuds_uniques

#si plus de 2 noeuds uniques, il faut regrouper
df_noeud_uniq=graph_filaire_123_vertex.loc[(graph_filaire_123_vertex['id'].isin(noeud_uniq))]
corresp_noeud_uniq=plus_proche_voisin(df_noeud_uniq, df_noeud_uniq, 100, 'id', 'id', True)

In [1205]:
corresp_noeud_uniq

,id_left,id_right
1,9383,4194
2,4194,9383
5,1034,1031
6,1031,1034


In [1206]:
#trouver tout les noeud si 2*2 voies : 
noeud=1034
#trouver le noeud supplementaire si besoin : 
list_tronc_noeud=gdf_rhv_cpt_perm_123.loc[(gdf_rhv_cpt_perm_123['source']==noeud) | (gdf_rhv_cpt_perm_123['target']==noeud)].copy().idtronc.tolist()
list_noeud_sup=[]
for t in list_tronc_noeud : 
    noeud_troncon=gdf_rhv_cpt_perm_123.loc[gdf_rhv_cpt_perm_123['idtronc']==t]
    noeud_uniq=[k for k,v in Counter(noeud_troncon.source.tolist()+noeud_troncon.target.tolist()).items() if v==1]
    if len(noeud_uniq)>2 : 
        df_noeud_uniq=graph_filaire_123_vertex.loc[(graph_filaire_123_vertex['id'].isin(noeud_uniq))]
        corresp_noeud_uniq=plus_proche_voisin(df_noeud_uniq, df_noeud_uniq, 30, 'id', 'id', True)
        list_noeud_sup+=corresp_noeud_uniq.loc[corresp_noeud_uniq['id_left']==noeud].id_right.tolist()
list_noeud_sup=list(set(list_noeud_sup))

In [1207]:
list_noeud_sup

[]